In [6]:
import glob
import numpy as np
import pandas as pd

In [7]:
def process_data(file_list, output_dir="./statistics", verbose=True, warning=True):
    """
    Reads LogPhase600 Excel files, computes AUC, maximal growth rate, maximal per-capita growth rate,
    lag time statistics and writes them to files -- one for each condition.
    
    Certain assumptions are made for the data -- the most important ones
    is that the number of wells per plate is 96 and that the sheets of the Excel file are named
    consistently.
    
    Parameter file_list: an array containing the paths to the Excel files.
    """
    
    !mkdir -p output_dir

    all_plate_numbers = []
    ref_wells = None
    ref_meta = None
    ref_strains = None
    ref_time = None

    def original_datetime_converter(x):
        t = str(x)
        if len(t) != 8:
            split_date = t.split(" ")
            day_mul = int(split_date[0].split("-")[-1])
            new_t = split_date[-1].split(":")
            new_t[0] = str(int(new_t[0]) + 24*day_mul)
            t = ":".join(new_t)
            if verbose: print("Replaced imported date", str(x), "with", t)
        return t

    if verbose: 
        print("Starting data import...")
        print("File list:")
        for f in file_list:
            print(f)
        print()

    for f in file_list:
        if verbose: print("Processing:", f)
        for pn in range(1, int(len(pd.ExcelFile(f).sheet_names)/3) + 1):
            if verbose: print("Plate", pn)
            plate_meta = pd.read_excel(f, sheet_name="Plate " + str(pn) + " - Results", header=1)
            plate_meta.iloc[:, 4] = plate_meta.iloc[:, 4].fillna("")
            plate_raw = pd.read_excel(f, sheet_name="Plate " + str(pn) + " - Raw Data", header=1, converters={"Time": original_datetime_converter})
            # plate_raw = plate_raw[["Time"] + list(plate_meta["Well"].values)]

            tn = np.array([np.round(int(x.split(":")[0]) + (int(x.split(":")[1])/60), 2) for x in plate_raw["Time"]])

            if ref_time is None:
                if verbose: print("Using", f, "time values as reference")
                ref_time = tn

            if ref_meta is None:
                if verbose: print("Using", f, "metadata columns as reference")
                ref_meta = plate_meta.columns.values

            if ref_wells is None:
                if verbose: print("Using", f, "raw data well names as reference")
                ref_wells = plate_raw.columns.values

            # Assertions
            if not np.all(ref_meta == plate_meta.columns.values) and warning:
                print("--------------------------------------------")
                print("Warning: in file", f, "plate", str(pn), "metadata columns do not match to the reference.")
                mismatch_idx = np.where(ref_meta != plate_meta.columns.values)[0]
                for mm in mismatch_idx:
                    print("    Column", plate_meta.columns.values[mm], "!= reference", ref_meta[mm])
                print("--------------------------------------------")

            if not np.all(ref_wells == plate_raw.columns.values) and warning:
                print("--------------------------------------------")
                print("Warning: in file", f, "plate", str(pn), "raw data columns do not match to the reference.")
                mismatch_idx = np.where(ref_wells != plate_raw.columns.values)[0]
                for mm in mismatch_idx:
                    print("    Column", plate_raw.columns.values[mm], "!= reference", ref_wells[mm])
                print("--------------------------------------------")

            if not np.all(ref_time == tn) and warning:
                print("--------------------------------------------")
                print("Warning: in file", f, "plate", str(pn), "time vector does not match to the reference.")
                print("--------------------------------------------")
                # assert 0

            assert np.all(np.array([len(str(x)) for x in plate_raw.Time]) == 8)
            assert np.all(plate_meta.iloc[:, 3] + " " + plate_meta.iloc[:, 4] == plate_meta.iloc[0, 3] + " " + plate_meta.iloc[0, 4])
            assert plate_meta.shape[0] == 96

            plate_meta = plate_meta.set_index("Well").loc[ref_wells[1:]].reset_index()
            
            
            plate_t = np.array([np.round(int(x.split(":")[0]) + (int(x.split(":")[1])/60), 2) for x in plate_raw.to_numpy().T[0, :]])
            plate_raw = plate_raw.to_numpy().T[1:, :]
            plate_meta = plate_meta.to_numpy()
            
            data = pd.DataFrame(columns=["Well", "Short strain ID", "Long strain ID", "Stressor", "Concentration", "AUC", "Maximal growth rate", "Maximal per-capita growth rate", "Lag time (hours)"])
            data["Well"] = plate_meta[:, 0].reshape(-1)
            data["Short strain ID"] = plate_meta[:, 1].reshape(-1)
            data["Long strain ID"] = plate_meta[:, 2].reshape(-1)
            data["Stressor"] = plate_meta[:, 3].reshape(-1)
            data["Concentration"] = plate_meta[:, 4].reshape(-1)

            m = plate_raw.astype(float)
            aucs = np.sum(m, axis=1)
            gr = m.copy()
            rho = m.copy()
            for i in range(m.shape[0]):
                gr[i, :] = np.array([0] + [(gr[i, j] - gr[i, j-1]) for j in range(1, m.shape[1])])
                rho[i, :] = np.array([0] + [(rho[i, j] - rho[i, j-1])/rho[i, j-1] if rho[i, j-1] != 0 else 0 for j in range(1, m.shape[1])])

            data["AUC"] = aucs
            data["Maximal growth rate"] = np.max(gr, axis=1)
            data["Maximal per-capita growth rate"] = np.max(rho, axis=1)
            data["Lag time (hours)"] = [plate_t[np.where(gr[i] == np.max(gr[i]))[0][0]] for i in range(len(gr))]
            
            data.to_csv(output_dir + "/" + data["Stressor"][0].replace(" ", "") + "_" + data["Concentration"][0].replace("/", "").replace(" ", "") + ".tsv", sep="\t", index=False)

        if verbose: print()

    if verbose: print("Done")
    

# File processing

### File list

In [8]:
files = glob.glob("./data/set_5_R6/*")  # Directory where the LogPhase Excel files are
files

['./data/set_5_R6/R6_CuSO4_1mM_1-2_H2O2_0,5mM_1-2_11-loka-2023 14-49-09.xlsx',
 './data/set_5_R6/R6_Cipro_1300ug(ml)_1-2_11-loka-2023 14-49-04.xlsx',
 './data/set_5_R6/R6_Azithro_25ug(ml)_1-2_Trimet_4000ug(ml)_1-2_11-loka-2023 14-48-52.xlsx',
 './data/set_5_R6/R6_Amxclv_25ug(ml)_1-2_NaCl_120mg(ml)_1-2_11-loka-2023 14-49-25.xlsx',
 './data/set_5_R6/R6_Control_1-2_Ceftrx_1300ug(ml)_1-2_11-loka-2023 14-49-16.xlsx']

### Reading files and writing statistics

In [9]:
process_data(files)  # This will create a folder "statistics" in the working directory and write the results there

Starting data import...
File list:
./data/set_5_R6/R6_CuSO4_1mM_1-2_H2O2_0,5mM_1-2_11-loka-2023 14-49-09.xlsx
./data/set_5_R6/R6_Cipro_1300ug(ml)_1-2_11-loka-2023 14-49-04.xlsx
./data/set_5_R6/R6_Azithro_25ug(ml)_1-2_Trimet_4000ug(ml)_1-2_11-loka-2023 14-48-52.xlsx
./data/set_5_R6/R6_Amxclv_25ug(ml)_1-2_NaCl_120mg(ml)_1-2_11-loka-2023 14-49-25.xlsx
./data/set_5_R6/R6_Control_1-2_Ceftrx_1300ug(ml)_1-2_11-loka-2023 14-49-16.xlsx

Processing: ./data/set_5_R6/R6_CuSO4_1mM_1-2_H2O2_0,5mM_1-2_11-loka-2023 14-49-09.xlsx
Plate 1
Replaced imported date 1900-01-01 00:00:02 with 24:00:02
Using ./data/set_5_R6/R6_CuSO4_1mM_1-2_H2O2_0,5mM_1-2_11-loka-2023 14-49-09.xlsx time values as reference
Using ./data/set_5_R6/R6_CuSO4_1mM_1-2_H2O2_0,5mM_1-2_11-loka-2023 14-49-09.xlsx metadata columns as reference
Using ./data/set_5_R6/R6_CuSO4_1mM_1-2_H2O2_0,5mM_1-2_11-loka-2023 14-49-09.xlsx raw data well names as reference
Plate 2
Replaced imported date 1900-01-01 00:00:02 with 24:00:02
Plate 3
Replaced imp